In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sickle_Cell_Anemia"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Sickle_Cell_Anemia/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Sickle_Cell_Anemia/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Sickle_Cell_Anemia/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Sickle_Cell_Anemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. Find the most relevant directory for Sickle Cell Anemia
subdirectories = os.listdir(tcga_root_dir)
target_trait = trait.lower().replace("_", " ")  # Convert to lowercase for case-insensitive matching

# Define key terms for Sickle Cell Anemia
key_terms = ["sickle", "anemia", "blood", "hematologic", "leukemia"]

# Start with no match, then find the best match based on similarity to target trait
best_match = None
best_match_score = 0
min_threshold = 2  # Require at least 2 matching terms or exact trait name match

for subdir in subdirectories:
    subdir_lower = subdir.lower()
    
    # Check for exact matches
    if target_trait in subdir_lower:
        best_match = subdir
        print(f"Found exact match: {subdir}")
        break
    
    # Calculate a score based on key terms
    score = 0
    for term in key_terms:
        if term in subdir_lower:
            score += 1
    
    # Check for partial matches with threshold
    if score > best_match_score and score >= min_threshold:
        best_match_score = score
        best_match = subdir
        print(f"Found potential match: {subdir} (score: {score})")

# Use the best match if found
if best_match:
    print(f"Selected directory: {best_match}")
    
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, best_match)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if we have both gene and trait data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0
    
else:
    print(f"No suitable directory found for {trait}. TCGA primarily contains cancer data, not blood disorders like Sickle Cell Anemia.")
    is_gene_available = False
    is_trait_available = False

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not best_match:
    print("Skipping this trait as no suitable data was found.")

No suitable directory found for Sickle_Cell_Anemia. TCGA primarily contains cancer data, not blood disorders like Sickle Cell Anemia.
Skipping this trait as no suitable data was found.
